# TF-IDF with Scikit-Learn

In the previous lesson, we learned about a text analysis method called *term frequency–inverse document frequency*, often abbreviated *tf-idf*. Tf-idf is a method that tries to identify the most distinctively frequent or significant words in a document. We specifically learned how to calculate tf-idf scores using word frequencies per page—or "extracted features"—made available by the HathiTrust Digital Library.

In this lesson, we're going to learn how to calculate tf-idf scores using a collection of plain text (.txt) files and the Python library scikit-learn, which has a quick and nifty module called [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html).

In this lesson, we will cover how to:
- Calculate and normalize tf-idf scores for U.S. Inaugural Addresses with scikit-learn

## Dataset

### U.S. Inaugural Addresses

<blockquote class="epigraph" style=" padding: 10px">

This is the meaning of our liberty and our creed; why men and women and children of every race and every faith can join in celebration across this magnificent Mall, and why a man whose father less than 60 years ago might not have been served at a local restaurant can now stand before you to take a most sacred oath.  So let us mark this day with remembrance of who we are and how far we have traveled.
<p class ="attribution">—Barack Obama, Inaugural Presidential Address, January 2009
    </p>
    
</blockquote>

During Barack Obama's Inaugural Address in January 2009, he mentioned "women" four different times, including in the passage quoted above. How distinctive is Obama's inclusion of women in this address compared to all other U.S. Presidents? This is one of the questions that we're going to try to answer with tf-idf.

## Breaking Down the TF-IDF Formula

But first, let's quickly discuss the tf-idf formula. The idea is pretty simple.

**tf-idf = term_frequency * inverse_document_frequency**

**term_frequency** = number of times a given term appears in document

**inverse_document_frequency** = log(total number of documents / number of documents with term) + 1**\***

You take the number of times a term occurs in a document (term frequency). Then you take the number of documents in which the same term occurs at least once divided by the total number of documents (document frequency), and you flip that fraction on its head (inverse document frequency). Then you multiply the two numbers together (term_frequency * inverse_document_frequency).

The reason we take the *inverse*, or flipped fraction, of document frequency is to boost the rarer words that occur in relatively few documents. Think about the inverse document frequency for the word "said" vs the word "pigeon." The term "said" appears in 13 (document frequency) of 14 (total documents) *Lost in the City* stories (14 / 13 --> a smaller inverse document frequency) while the term "pigeons" only occurs in 2 (document frequency) of the 14 stories (total documents) (14 / 2 --> a bigger inverse document frequency, a bigger tf-idf boost). 

*There are a bunch of slightly different ways that you can calculate inverse document frequency. The version of idf that we're going to use is the [scikit-learn default](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer), which uses "smoothing" aka it adds a "1" to the numerator and denominator: 

**inverse_document_frequency**  = log((1 + total_number_of_documents) / (number_of_documents_with_term +1)) + 1

<div class="margin sidebar" style=" padding: 10px">

> If smooth_idf=True (the default), the constant “1” is added to the numerator and denominator of the idf as if an extra document was seen containing every term in the collection exactly once, which prevents zero divisions: idf(t) = log [ (1 + n) / (1 + df(t)) ] + 1.  
> -[scikit-learn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer)

</div>

## TF-IDF with scikit-learn

[scikit-learn](https://scikit-learn.org/stable/index.html), imported as `sklearn`, is a popular Python library for machine learning approaches such as clustering, classification, and regression. Though we're not doing any machine learning in this lesson, we're nevertheless going to use scikit-learn's `TfidfVectorizer` and `CountVectorizer`.

Install scikit-learn

In [1]:
!pip install sklearn

  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=bcb5b28adb5f8d4f3e31c26dba69f14f7a30c1f5585d6b5403d5d826cd574072
  Stored in directory: /Users/cblevins/Library/Caches/pip/wheels/f8/e0/3d/9d0c2020c44a519b9f02ab4fa6d2a4a996c98d79ab2f569fa1
Successfully built sklearn


Import necessary modules and libraries

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
pd.set_option("max_rows", 600)
from pathlib import Path  
import glob

<div class="admonition pandasreview" name="html-admonition" style="background: black; color: white; padding: 10px">
<p class="title">Pandas</p>
 Do you need a refresher or introduction to the Python data analysis library Pandas? Be sure to check out <a href="https://melaniewalsh.github.io/Intro-Cultural-Analytics/Data-Analysis/Pandas-Basics-Part1.html"> Pandas Basics (1-3) </a> in this textbook!
    
</div>

We're also going to import `pandas` and change its default display setting. And we're going to import two libraries that will help us work with files and the file system: [`pathlib`](https://docs.python.org/3/library/pathlib.html##basic-use) and [`glob`](https://docs.python.org/3/library/glob.html).

#### Set Directory Path

Below we're setting the directory filepath that contains all the text files that we want to analyze.

In [2]:
directory_path = "./elohp-interview-text/"

Then we're going to use `glob` and `Path` to make a list of all the filepaths in that directory and a list of all the short story titles.

In [3]:
text_files = glob.glob(f"{directory_path}/*.txt")

In [4]:
text_files

['./elohp-interview-text/Coll520_do065_interviewtext.txt',
 './elohp-interview-text/Coll520_do003_interviewtext.txt',
 './elohp-interview-text/Coll520_do051_interviewtext.txt',
 './elohp-interview-text/Coll520_do060_interviewtext.txt',
 './elohp-interview-text/Coll520_do048_interviewtext.txt',
 './elohp-interview-text/Coll520_do006_interviewtext.txt',
 './elohp-interview-text/Coll520_do054_interviewtext.txt',
 './elohp-interview-text/Coll520_do032_interviewtext.txt',
 './elohp-interview-text/Coll520_do002_interviewtext.txt',
 './elohp-interview-text/Coll520_do036_interviewtext.txt',
 './elohp-interview-text/Coll520_do061_interviewtext.txt',
 './elohp-interview-text/Coll520_do007_interviewtext.txt',
 './elohp-interview-text/Coll520_do049_interviewtext.txt',
 './elohp-interview-text/Coll520_do055_interviewtext.txt',
 './elohp-interview-text/Coll520_do033_interviewtext.txt',
 './elohp-interview-text/Coll520_do029_interviewtext.txt',
 './elohp-interview-text/Coll520_do067_interviewtext.txt

In [5]:
text_titles = [Path(text).stem for text in text_files]

In [6]:
text_titles

['Coll520_do065_interviewtext',
 'Coll520_do003_interviewtext',
 'Coll520_do051_interviewtext',
 'Coll520_do060_interviewtext',
 'Coll520_do048_interviewtext',
 'Coll520_do006_interviewtext',
 'Coll520_do054_interviewtext',
 'Coll520_do032_interviewtext',
 'Coll520_do002_interviewtext',
 'Coll520_do036_interviewtext',
 'Coll520_do061_interviewtext',
 'Coll520_do007_interviewtext',
 'Coll520_do049_interviewtext',
 'Coll520_do055_interviewtext',
 'Coll520_do033_interviewtext',
 'Coll520_do029_interviewtext',
 'Coll520_do067_interviewtext',
 'Coll520_do001_interviewtext',
 'Coll520_do053_interviewtext',
 'Coll520_do035_interviewtext',
 'Coll520_do062_interviewtext',
 'Coll520_do004_interviewtext',
 'Coll520_do018_interviewtext',
 'Coll520_do056_interviewtext',
 'Coll520_do030_interviewtext',
 'Coll520_do066_interviewtext',
 'Coll520_do028_interviewtext',
 'Coll520_do052_interviewtext',
 'Coll520_do034_interviewtext',
 'Coll520_do063_interviewtext',
 'Coll520_do005_interviewtext',
 'Coll52

## Calculate tf–idf

To calculate tf–idf scores for every word, we're going to use scikit-learn's [`TfidfVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html).

When you initialize TfidfVectorizer, you can choose to set it with different parameters. These parameters will change the way you calculate tf–idf.

The recommended way to run `TfidfVectorizer` is with smoothing (`smooth_idf = True`) and normalization (`norm='l2'`) turned on. These parameters will better account for differences in text length, and overall produce more meaningful tf–idf scores. Smoothing and L2 normalization are actually the default settings for `TfidfVectorizer`, so to turn them on, you don't need to include any extra code at all.

Initialize TfidfVectorizer with desired parameters (default smoothing and normalization)

In [7]:
from nltk.corpus import stopwords
stop = list(stopwords.words('english'))
stop.extend('raiskin myword2 myword3'.split())
vectorizer = TfidfVectorizer(analyzer = 'word',stop_words=set(stop))
#vectors = vectorizer.fit_transform(corpus)

In [8]:
#tfidf_vectorizer = TfidfVectorizer(input='filename', stop_words='english')
tfidf_vectorizer = TfidfVectorizer(input='filename', stop_words=set(stop))

Run TfidfVectorizer on our `text_files`

In [9]:
tfidf_vector = tfidf_vectorizer.fit_transform(text_files)

Make a DataFrame out of the resulting tf–idf vector, setting the "feature names" or words as columns and the titles as rows

In [10]:
tfidf_df = pd.DataFrame(tfidf_vector.toarray(), index=text_titles, columns=tfidf_vectorizer.get_feature_names())

Add column for document frequency aka number of times word appears in all documents

In [11]:
tfidf_df.loc['00_Document Frequency'] = (tfidf_df > 0).sum()

In [12]:
tfidf_slice = tfidf_df[['lesbian', 'gay']]
tfidf_slice.sort_index().round(decimals=2)

,lesbian,gay
00_Document Frequency,71.00,70.00
Coll520_do001_interviewtext,0.06,0.03
Coll520_do002_interviewtext,0.02,0.04
Coll520_do003_interviewtext,0.06,0.05
Coll520_do004_interviewtext,0.02,0.01
Coll520_do005_interviewtext,0.07,0.04
Coll520_do006_interviewtext,0.05,0.03
Coll520_do007_interviewtext,0.13,0.08
Coll520_do008_interviewtext,0.05,0.03
Coll520_do009_interviewtext,0.05,0.02


Let's drop "OO_Document Frequency" since we were just using it for illustration purposes.

In [13]:
tfidf_df = tfidf_df.drop('00_Document Frequency', errors='ignore')

Let's reorganize the DataFrame so that the words are in rows rather than columns.

In [14]:
tfidf_df.stack().reset_index()

,level_0,level_1,0
0,Coll520_do065_interviewtext,00,0.000000
1,Coll520_do065_interviewtext,000,0.003431
2,Coll520_do065_interviewtext,00am,0.000000
3,Coll520_do065_interviewtext,00pm,0.000000
4,Coll520_do065_interviewtext,00s,0.000000
...,...,...,...
1264931,Coll520_do058_interviewtext,zoomed,0.000000
1264932,Coll520_do058_interviewtext,zukowski,0.000000
1264933,Coll520_do058_interviewtext,zumba,0.000000
1264934,Coll520_do058_interviewtext,zweig,0.000000


In [15]:
tfidf_df = tfidf_df.stack().reset_index()

In [16]:
tfidf_df = tfidf_df.rename(columns={0:'tfidf', 'level_0': 'document','level_1': 'term', 'level_2': 'term'})

To find out the top 10 words with the highest tf–idf for every story, we're going to sort by document and tfidf score and then groupby document and take the first 10 values.

In [17]:
tfidf_df.sort_values(by=['document','tfidf'], ascending=[True,False]).groupby(['document']).head(10)

,document,term,tfidf
303463,Coll520_do001_interviewtext,agapito,0.610865
320455,Coll520_do001_interviewtext,womanshare,0.169570
315874,Coll520_do001_interviewtext,really,0.142410
320529,Coll520_do001_interviewtext,would,0.138613
316153,Coll520_do001_interviewtext,remember,0.131018
...,...,...,...
989137,Coll520_do074_interviewtext,lesbian,0.095365
986504,Coll520_do074_interviewtext,fun,0.082813
986822,Coll520_do074_interviewtext,going,0.078336
988784,Coll520_do074_interviewtext,kind,0.078336


In [18]:
top_tfidf = tfidf_df.sort_values(by=['document','tfidf'], ascending=[True,False]).groupby(['document']).head(10)

We can zoom in on particular words and particular documents.

In [19]:
top_tfidf[top_tfidf['term'].str.contains('women')]

,document,term,tfidf
320459,Coll520_do001_interviewtext,women,0.125321
35403,Coll520_do003_interviewtext,women,0.089180
391723,Coll520_do004_interviewtext,women,0.080173
552068,Coll520_do005_interviewtext,womenspace,0.098685
872755,Coll520_do008_interviewtext,women,0.188202
765859,Coll520_do015_interviewtext,women,0.073654
1246891,Coll520_do016_interviewtext,women,0.116463
409539,Coll520_do018_interviewtext,women,0.079317
890571,Coll520_do020_interviewtext,women,0.081946
730227,Coll520_do021_interviewtext,women,0.131960


It turns out that the term "women" is very distinctive in Obama's Inaugural Address.

In [20]:
top_tfidf[top_tfidf['document'].str.contains('kennedy')]

,document,term,tfidf


## Visualize TF-IDF

We can also visualize our TF-IDF results with the data visualization library Altair.

In [21]:
#!pip install altair

Let's make a heatmap that shows the highest TF-IDF scoring words for each president, and let's put a red dot next to two terms of interest: "war" and "peace":

The code below was contributed by [Eric Monson](https://github.com/emonson). Thanks, Eric!

In [22]:
import altair as alt
import numpy as np

# Terms in this list will get a red dot in the visualization
term_list = ['war', 'peace']

# adding a little randomness to break ties in term ranking
top_tfidf_plusRand = top_tfidf.copy()
top_tfidf_plusRand['tfidf'] = top_tfidf_plusRand['tfidf'] + np.random.rand(top_tfidf.shape[0])*0.0001

# base for all visualizations, with rank calculation
base = alt.Chart(top_tfidf_plusRand).encode(
    x = 'rank:O',
    y = 'document:N'
).transform_window(
    rank = "rank()",
    sort = [alt.SortField("tfidf", order="descending")],
    groupby = ["document"],
)

# heatmap specification
heatmap = base.mark_rect().encode(
    color = 'tfidf:Q'
)

# red circle over terms in above list
circle = base.mark_circle(size=100).encode(
    color = alt.condition(
        alt.FieldOneOfPredicate(field='term', oneOf=term_list),
        alt.value('red'),
        alt.value('#FFFFFF00')        
    )
)

# text labels, white for darker heatmap colors
text = base.mark_text(baseline='middle').encode(
    text = 'term:N',
    color = alt.condition(alt.datum.tfidf >= 0.23, alt.value('white'), alt.value('black'))
)

# display the three superimposed visualizations
(heatmap + circle + text).properties(width = 600)

alt.LayerChart(...)

## Your Turn!

Take a few minutes to explore the dataframe below and then answer the following questions.

**1.** What is the difference between a tf-idf score and raw word frequency?

**Your answer here**

**2.** Based on the dataframe above, what is one potential problem or limitation that you notice with tf-idf scores?

**Your answer here**

**3.** What's another collection of texts that you think might be interesting to analyze with tf-idf scores?  Why?

**Your answer here**